## EU Driving regulation for truckers

The EU Regulation 561/2006 sets rules for driving time, breaks, and rest periods for drivers of goods vehicles over 3.5 tons. These regulations aim to improve road safety and protect drivers from excessive fatigue. This is a critical problem in the logistic industry that is actually handled by humans, manually defining the routes and breaks for the drivers. ...

Maximun Driving Time:
* Driving: 4.5 hours of continuous driving.
* Daily: 9 hours, extendable to 10 hours twice a week.
* Weekly: 56 hours.
* Bi-weekly: 90 hours in any two consecutive weeks.

Breaks:

* Driving: 45 minutes is mandatory (can be split into 15 minutes followed by 30 minutes) After 4.5 hours of continuous driving.
* Daily rest: 11 hours or reduced 9 hours (up to three times per week). After 9 hours of driving.
* Weekly rest: 45 hours or a reduced weekly rest period of at least 24 hours.



The tachograph records various driving and vehicle data, ensuring compliance with driving time, breaks, and rest period regulations. Information recorded by a tachograph includes:

* Driver's details: The driver's name, license number, and tachograph card number.
* Date and time: The device records date and time for all activities.
* Vehicle speed: The tachograph constantly tracks and records the vehicle's speed.
* Driving time: The device records the duration of driving sessions, including any breaks or rest periods.
* Distance traveled: The tachograph logs the distance covered during a driving session.
* Breaks and rest periods: The device records the duration and frequency of breaks and rest periods taken by the driver.
* Work mode: The tachograph differentiates between driving, working, availability, and rest modes.
* Events and faults: The device logs any events or faults, such as overspeeding, power supply interruptions, or sensor malfunctions.

# Example of a tachograph

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/driver1.csv")
status_translation = {
    'Pausa': 'Break',
    'Conduciendo': 'Driving',
    'Otro trabajo': 'Other work',
    'En espera': 'On standby'
}

# Replace the values in the Status column with the translated values
df['Status'] = df['Status'].map(status_translation)
df.head()

,Driver,DateStart,HourStart,HourEnd,Duration,Distance,Status,Profession,Vehicle,PlaceStart,PlaceEnd,LatitudeStart,LongitudeStart,LatitudeEnd,LongitudeEnd,MilometerStart,MilometerEnd
0,driver1,lun 2/01/17,17:59,18:13,0h 14m,0.028144,Break,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327719,-3.784834,40.327719,-3.784502,"31,512","31,512"
1,driver1,lun 2/01/17,18:13,18:16,0h 03m,0.124866,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.327926,-3.784502,40.327926,-3.785975,"31,512","31,513"
2,driver1,lun 2/01/17,18:16,18:18,0h 02m,0.000509,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.329214,-3.785975,40.329214,-3.785981,"31,513","31,513"
3,driver1,lun 2/01/17,18:18,18:20,0h 02m,0.230232,Driving,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328715,-3.785981,40.328715,-3.788697,"31,513","31,513"
4,driver1,lun 2/01/17,18:20,18:43,0h 23m,0.001611,Other work,1,9359JSNMI6186H01502,"28914, Leganés, ES","28914, Leganés, ES",40.328620,-3.788697,40.328620,-3.788678,"31,513","31,513"


In [28]:
df['Status'].unique()

array(['Break', 'Driving', 'Other work', 'On standby'], dtype=object)

# Solving it with UPF

####  _______ Begin Installation____________

**Disclaimer** : The installation steps are only needed until up_siadex is published on pypi and the changes on the unified-planning fork are merged into the origin package

In [ ]:
# Make sure the following packages are installed on the system: python-dev libreadline-dev. Those are needed for the execution of Siadex
!apt-get update
!apt-get install -y python-dev libreadline-dev

In [ ]:
# Clonning the repos
!git clone https://github.com/UGR-IntelligentSystemsGroup/unified-planning.git
!git clone https://github.com/UGR-IntelligentSystemsGroup/up-siadex.git

In [ ]:
# Install the packages
# %%capture
%pip install ./unified-planning
%pip install ./up-siadex

####  _______ End Installation____________

In [4]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


In [5]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

In [11]:
reader = HPDLReader()
problem = reader.parse_problem("./unified-planning/unified_planning/test/pddl/hpdl/jacob/domain.hpdl","./unified-planning/unified_planning/test/pddl/hpdl/jacob/problem.hpdl")

# A look to the problem 

In [12]:
action_status = {
"d_p": "Driving",
"o_p": "Other Work",
"b_p": "Break",
"i_p": "Idle",
"load_p": "Load Package",
"unload_p": "Unload Package",
"drive_p": "Drive to Destination",
"drive_p_fixed_duration": "Drive with Fixed Duration",
"refuel": "Refuel",
"refuel-on-the-way": "Refuel on the Way",
"d_suggested": "Suggested Driving",
"b_suggested": "Suggested Break"
}

In [13]:
def create_tree(task_translation, method_task_translation):
    tree = {}

    for (method, task), translation in method_task_translation.items():
        task_name = task_translation[task]
        if task_name not in tree:
            tree[task_name] = {}
        tree[task_name][method] = translation

    return tree


def print_tree(tree):
    print(f"___________________\n")
    for task_name, methods in tree.items():
        print(f"Task: [{task_name}]:")
        print(f"Methods:")
        for method, translation in methods.items():
            print(f"  - {translation}")
        print(f"___________________\n")
    print(f"Actions:")
    for action,translation in action_status.items():
        print(f"- {translation}")


tree = create_tree(task_translation, method_task_translation)
print_tree(tree)

___________________

Task: [Reset Counters]:
Methods:
  - Reset Counters (Single)
___________________

Task: [Weekly Driving]:
Methods:
  - Weekly Driving (Weekly Rest)
  - Weekly Driving (Normal)
  - Weekly Driving (Illegal Biweek)
  - Weekly Driving (Transport)
  - Weekly Driving (End)
___________________

Task: [Daily Driving]:
Methods:
  - Daily Driving (Normal Daily Driving)
  - Daily Driving (Extended Daily Driving)
  - Daily Driving (Continuous Daily Driving)
  - Daily Driving (Rest Day)
  - Daily Driving (Ignore Action)
  - Daily Driving (End)
___________________

Task: [Ignore Activity]:
Methods:
  - Ignore Activity (Activity)
  - Ignore Activity (Unknown Break)
___________________

Task: [Normal Daily Driving]:
Methods:
  - Normal Daily Driving (Two Continuous Daily Drivings)
  - Normal Daily Driving (Unique)
___________________

Task: [Extended Daily Driving]:
Methods:
  - Extended Daily Driving (Extended Daily Driving)
___________________

Task: [Continuous Daily Driving]:


# Solving with Siadex

In [15]:
with env.factory.OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    print(result)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.timed_actions):
            action_name = a[1].action.name
            if a[1].action.name in action_status:
                action_name = action_status[action_name]
            print(f"""Start: {a[0]} - End: {a[0] + a[2]} \n {action_name}({a[1].actual_parameters})""")
            print(f"_"*50)
    else:
        print('No plan found!')

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_2230/2884070800.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



/mnt/e/newDesktop/void/projects/siadex/up-siadex/Notebooks/unified-planning/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7fc0bc81d3f0> cannot solve this kind of problem!
  warn(msg)


SIADEX found a valid plan!
The plan is: 

Start: 0.0 - End: 5.0 
 Break((driver, '5', b_t0, split_1, first, ndd, '1', '1', yes))
__________________________________________________
Start: 5.0 - End: 56.0 
 Driving((driver, '51', a, split_1, first, ndd, '1', '1', yes))
__________________________________________________
Start: 56.0 - End: 193.0 
 Break((driver, '137', b_t2, split_1, first, ndd, '1', '1', yes))
__________________________________________________
Start: 193.0 - End: 195.0 
 Driving((driver, '2', a, split_2, first, ndd, '1', '1', yes))
__________________________________________________
Start: 195.0 - End: 198.0 
 Other Work((driver, '3', a, split_2, first, ndd, '1', '1', yes))
__________________________________________________
Start: 198.0 - End: 200.0 
 Driving((driver, '2', a, split_2, first, ndd, '1', '1', yes))
__________________________________________________
Start: 200.0 - End: 202.0 
 Other Work((driver, '2', a, split_2, first, ndd, '1', '1', yes))
___________________